In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load Datasets

## LEIE

In [9]:
exclusion_dict = {"NPI": int, "GENERAL": str, "SPECIALTY": str, "CITY": str, "STATE": str}
exclusion_df = pd.read_csv("/content/drive/MyDrive/BT4012 Project/Submission Folder/Data_2019/LEIE.csv", 
                           encoding='latin-1', usecols = exclusion_dict.keys(), dtype = exclusion_dict)

In [10]:
exclusion_df

,GENERAL,SPECIALTY,NPI,CITY,STATE
0,OTHER BUSINESS,SOBER HOME,0,BROOKLYN,NY
1,OTHER BUSINESS,PHARMACY,1972902351,NEW YORK,NY
2,PHARMACY,NaN,0,SMITHTOWN,NY
3,DME COMPANY,DME - OXYGEN,0,MIAMI,FL
4,OTHER BUSINESS,PHARMACY,1922348218,BRONX,NY
...,...,...,...,...,...
76405,IND- LIC HC SERV PRO,NURSE/NURSES AIDE,0,COAL TOWNSHIP,PA
76406,PHARMACY,TECHNICIAN,0,RUCKERSVILLE,VA
76407,NURSING PROFESSION,NURSE/NURSES AIDE,0,KANSAS CITY,MO
76408,REHAB FACILITY - GEN,EMPLOYEE,0,ST CLOUD,MN


In [11]:
fraudster_list = exclusion_df.loc[:,['NPI']]
fraudster_list = fraudster_list[~(fraudster_list["NPI"]==0)]
fraudster_list = fraudster_list.rename(columns = {'NPI':'npi'})
fraudster_list["is_fraud"] = 1
fraudster_list.head()

,npi,is_fraud
1,1972902351,1
4,1922348218,1
24,1942476080,1
28,1275600959,1
31,1891731758,1


In [12]:
len(fraudster_list)

6384

## Part B

### Load Data

* **Rndrng_NPI**: National Provider Identifier
* **Rndrng_Prvdr_Crdntls**: Credentials of the Provider
* **Rndrng_Prvdr_Type**: Provider Type of the Provider
* **Rndrng_Prvdr_Gndr**: Gender of the Provider
* **Tot_Benes**: Number of Distinct Medicare Beneficiaries
* **Tot_Srvcs**: Number of Services
* **Tot_Bene_Day_Srvcs**: Number of Distinct Medicare Beneficiary/Per Day Services
* **Avg_Sbmtd_Chrg**: Average Submitted Charge Amount
* **Avg_Mdcr_Alowd_Amt**: Average Medicare Allowed Amount
* **Avg_Mdcr_Pymt_Amt**: Average Medicare Payment Amount

In [71]:
part_b_data_dict = {"Rndrng_NPI": int, "Rndrng_Prvdr_Type": str, "Rndrng_Prvdr_Gndr": str, "Tot_Benes": "Int64", 
                    "Tot_Srvcs": "float64", "Tot_Bene_Day_Srvcs": "Int64", "Avg_Sbmtd_Chrg": "float64", 
                    "Avg_Mdcr_Alowd_Amt": "float64", "Avg_Mdcr_Pymt_Amt": "float64"}
part_b_df = pd.read_csv("/content/drive/MyDrive/BT4012 Project/Submission Folder/Data_2019/partB-2019.csv", 
                        encoding='latin-1', usecols = part_b_data_dict.keys(), dtype = part_b_data_dict)
part_b_df.head()

,Rndrng_NPI,Rndrng_Prvdr_Gndr,Rndrng_Prvdr_Type,Tot_Benes,Tot_Srvcs,Tot_Bene_Day_Srvcs,Avg_Sbmtd_Chrg,Avg_Mdcr_Alowd_Amt,Avg_Mdcr_Pymt_Amt
0,1003000126,M,Internal Medicine,40,40.0,40,232.275000,72.59000,57.870000
1,1003000126,M,Internal Medicine,25,25.0,25,712.800000,186.72520,148.851600
2,1003000126,M,Internal Medicine,24,24.0,24,320.166667,101.12125,80.615833
3,1003000126,M,Internal Medicine,157,158.0,158,651.417722,200.93000,160.190000
4,1003000126,M,Internal Medicine,117,205.0,205,245.614634,72.75639,58.005561


In [72]:
part_b_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10140228 entries, 0 to 10140227
Data columns (total 9 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Rndrng_NPI          int64  
 1   Rndrng_Prvdr_Gndr   object 
 2   Rndrng_Prvdr_Type   object 
 3   Tot_Benes           Int64  
 4   Tot_Srvcs           float64
 5   Tot_Bene_Day_Srvcs  Int64  
 6   Avg_Sbmtd_Chrg      float64
 7   Avg_Mdcr_Alowd_Amt  float64
 8   Avg_Mdcr_Pymt_Amt   float64
dtypes: Int64(2), float64(4), int64(1), object(2)
memory usage: 715.6+ MB


In [73]:
# count of npi values in Part B dataset
print("Number of unique npi ids:", part_b_df["Rndrng_NPI"].nunique())

Number of unique npi ids: 1093367


In [74]:
part_b_df["Rndrng_Prvdr_Gndr"].value_counts()

M    6510394
F    3193829
Name: Rndrng_Prvdr_Gndr, dtype: int64

In [75]:
print("Number of unique provider type:", part_b_df["Rndrng_Prvdr_Type"].nunique())

Number of unique provider type: 99


### Data Preprocessing

In [76]:
part_b_features = part_b_df.groupby(['Rndrng_NPI', 'Rndrng_Prvdr_Gndr', 'Rndrng_Prvdr_Type'])\
  .agg({'Tot_Benes':['mean'],
        'Tot_Srvcs':['mean'],
        'Tot_Bene_Day_Srvcs':['mean'],
        'Avg_Sbmtd_Chrg':['mean'],
        'Avg_Mdcr_Alowd_Amt':['mean'],
        'Avg_Mdcr_Pymt_Amt':['mean']
       })
part_b_features.columns = [col[0]+"_Mean" for col in part_b_features.columns.values]
part_b_features = part_b_features.reset_index(level=['Rndrng_NPI', 'Rndrng_Prvdr_Gndr', 'Rndrng_Prvdr_Type'])

In [77]:
part_b_features = part_b_features.merge(fraudster_list, how="left", left_on="Rndrng_NPI", right_on="npi")
part_b_features["is_fraud"] = part_b_features["is_fraud"].fillna(0)
part_b_features = part_b_features.rename(columns={"is_fraud": "Is_Fraud"})
part_b_features = part_b_features.drop(columns=["npi"])
part_b_features = part_b_features.rename(columns={"Rndrng_NPI": "NPI"})

In [78]:
print(part_b_features["Is_Fraud"].value_counts())
print(part_b_features["Is_Fraud"].value_counts(dropna=False, normalize=True))

0.0    1033745
1.0        220
Name: Is_Fraud, dtype: int64
0.0    0.999787
1.0    0.000213
Name: Is_Fraud, dtype: float64


In [79]:
part_b_features.head()

,NPI,Rndrng_Prvdr_Gndr,Rndrng_Prvdr_Type,Tot_Benes_Mean,Tot_Srvcs_Mean,Tot_Bene_Day_Srvcs_Mean,Avg_Sbmtd_Chrg_Mean,Avg_Mdcr_Alowd_Amt_Mean,Avg_Mdcr_Pymt_Amt_Mean,Is_Fraud
0,1003000126,M,Internal Medicine,107.666667,151.888889,151.888889,420.854343,126.457125,100.817897,0.0
1,1003000134,M,Pathology,397.75,650.416667,478.5,148.833333,35.661301,28.027689,0.0
2,1003000142,M,Anesthesiology,39.388889,60.833333,58.388889,184.386466,98.968066,72.129892,0.0
3,1003000423,F,Obstetrics & Gynecology,27.333333,28.666667,28.666667,63.333333,49.737654,44.211358,0.0
4,1003000480,M,General Surgery,24.333333,27.666667,27.666667,174.666667,42.652947,29.135261,0.0


In [80]:
part_b_features.isnull().sum()

NPI                        0
Rndrng_Prvdr_Gndr          0
Rndrng_Prvdr_Type          0
Tot_Benes_Mean             0
Tot_Srvcs_Mean             0
Tot_Bene_Day_Srvcs_Mean    0
Avg_Sbmtd_Chrg_Mean        0
Avg_Mdcr_Alowd_Amt_Mean    0
Avg_Mdcr_Pymt_Amt_Mean     0
Is_Fraud                   0
dtype: int64

In [81]:
#OneHotEncode Rfrg_Prvdr_Gndr
part_b_features['Is_Male'] = np.where(part_b_features['Rndrng_Prvdr_Gndr']=='M', 1.0, 0.0)
part_b_features = part_b_features.drop(['Rndrng_Prvdr_Gndr'], axis=1)

part_b_features.head()

,NPI,Rndrng_Prvdr_Type,Tot_Benes_Mean,Tot_Srvcs_Mean,Tot_Bene_Day_Srvcs_Mean,Avg_Sbmtd_Chrg_Mean,Avg_Mdcr_Alowd_Amt_Mean,Avg_Mdcr_Pymt_Amt_Mean,Is_Fraud,Is_Male
0,1003000126,Internal Medicine,107.666667,151.888889,151.888889,420.854343,126.457125,100.817897,0.0,1.0
1,1003000134,Pathology,397.75,650.416667,478.5,148.833333,35.661301,28.027689,0.0,1.0
2,1003000142,Anesthesiology,39.388889,60.833333,58.388889,184.386466,98.968066,72.129892,0.0,1.0
3,1003000423,Obstetrics & Gynecology,27.333333,28.666667,28.666667,63.333333,49.737654,44.211358,0.0,0.0
4,1003000480,General Surgery,24.333333,27.666667,27.666667,174.666667,42.652947,29.135261,0.0,1.0


### Descriptive Statistics

In [82]:
column_names = part_b_features.drop(['NPI', 'Is_Fraud', 'Is_Male', 'Rndrng_Prvdr_Type'], axis=1).columns.values.tolist() # Columns stats without meaning
descriptive_stats = part_b_features.drop(['NPI', 'Is_Fraud', 'Is_Male', 'Rndrng_Prvdr_Type'], axis=1).describe().loc[['mean', 'std', '50%']].transpose()
descriptive_stats.columns = ['mean', 'std', 'median']
descriptive_stats['mode'] = part_b_features.drop(['NPI', 'Is_Fraud', 'Is_Male', 'Rndrng_Prvdr_Type'], axis=1).mode().transpose()[0]
descriptive_stats['mad'] = [stats.median_abs_deviation(part_b_features[x]) for x in column_names]
descriptive_stats['kurtosis'] = [stats.kurtosis(part_b_features[x], fisher=False) for x in column_names]
descriptive_stats['excess kurtosis'] = [stats.kurtosis(part_b_features[x], fisher=True) for x in column_names]
descriptive_stats['skewness'] = [stats.skew(part_b_features[x]) for x in column_names]
descriptive_stats

,mean,std,median,mode,mad,kurtosis,excess kurtosis,skewness
Tot_Benes_Mean,56.872288,55.122156,43.789474,11.0,21.071637,784.000528,781.000528,13.898170
Tot_Srvcs_Mean,153.328471,474.477850,77.500000,12.0,46.700000,32181.854156,32178.854156,115.312049
Tot_Bene_Day_Srvcs_Mean,113.728663,160.181643,71.250000,12.0,40.535714,309.469120,306.469120,9.545144
Avg_Sbmtd_Chrg_Mean,363.030316,565.471102,193.714175,50.0,102.586256,294.780421,291.780421,10.382454
Avg_Mdcr_Alowd_Amt_Mean,94.584074,77.625741,81.828494,10.57,30.978783,361.743762,358.743762,10.167852
Avg_Mdcr_Pymt_Amt_Mean,72.684422,61.873906,61.881035,10.57,24.401117,362.479899,359.479899,10.218969


### Cleanup

In [83]:
del part_b_df

## DMEPOS

### Load Data

* **Rfrg_NPI**: National Provider Identifier (NPI) for the referring provider on the DMEPOS claim.
* **Rfrg_Prvdr_Type**: Derived from the Medicare provider/supplier specialty code reported on all of the NPI's Part B non-institutional claims
* **Tot_Suplrs**: Number of suppliers rendering DMEPOS products/services ordered by the referring provider.
* **Tot_Suplr_Benes**: Number of beneficiaries associated with the supplier DMEPOS products/services ordered by the referring provider. Beneficiary counts fewer than 11 have been suppressed to protect the privacy of Medicare beneficiaries.
* **Tot_Suplr_Clms**: Number of DMEPOS claims submitted by the supplier, reflecting products/services ordered by the referring provider. Aggregated records based on number_of_supplier_claims fewer than 11 are not included in the data file.
* **Tot_Suplr_Srvcs**: Number of DMEPOS products/services rendered by the supplier; note that the metrics used to count the number provided can vary from service to service.
* **Avg_Suplr_Sbmtd_Chrg**: Average of the charges that suppliers submit for DMEPOS products/services. Total submitted charges can be calculated by multiplying the avg_supplier_submitted_charge by the number_of_supplier_services.
* **Avg_Suplr_Mdcr_Alowd_Amt**: Average Medicare allowed amounts for the DMEPOS product/service rendered by suppliers. Medicare allowed amounts includes the amount Medicare pays, the deductible and coinsurance amounts that the beneficiary is responsible for paying, and any amounts that a third party is responsible for paying. Total Medicare allowed amounts can be calculated by multiplying the avg_supplier_medicare_allow_amt by the number_of_supplier_services.

In [84]:
dmepos_data_dict = {
"Rfrg_NPI": int, "Rfrg_Prvdr_Type": str, "Rfrg_Prvdr_Gndr": str,
"Tot_Suplrs": "float64", "Tot_Suplr_Benes": "float64", "Tot_Suplr_Clms": "float64", "Tot_Suplr_Srvcs": "float64",
"Avg_Suplr_Sbmtd_Chrg": "float64", "Avg_Suplr_Mdcr_Alowd_Amt": "float64"
}
dmepos_df = pd.read_csv("/content/drive/MyDrive/BT4012 Project/Submission Folder/Data_2019/DMEPOS-2019.csv", 
                        encoding='latin-1', usecols = dmepos_data_dict.keys(), dtype = dmepos_data_dict)
dmepos_df.head()

,Rfrg_NPI,Rfrg_Prvdr_Gndr,Rfrg_Prvdr_Type,Tot_Suplrs,Tot_Suplr_Benes,Tot_Suplr_Clms,Tot_Suplr_Srvcs,Avg_Suplr_Sbmtd_Chrg,Avg_Suplr_Mdcr_Alowd_Amt
0,1003000126,M,Internal Medicine,5.0,NaN,69.0,69.0,52.207536,22.961739
1,1003000126,M,Internal Medicine,5.0,12.0,88.0,88.0,362.305227,128.407955
2,1003000522,M,Family Practice,15.0,22.0,55.0,132.0,61.314242,8.364545
3,1003000522,M,Family Practice,2.0,NaN,13.0,13.0,14.158462,3.429231
4,1003000522,M,Family Practice,8.0,13.0,22.0,30.0,13.608667,1.436000


In [85]:
dmepos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1656449 entries, 0 to 1656448
Data columns (total 9 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   Rfrg_NPI                  1656449 non-null  int64  
 1   Rfrg_Prvdr_Gndr           1656403 non-null  object 
 2   Rfrg_Prvdr_Type           1656449 non-null  object 
 3   Tot_Suplrs                1656449 non-null  float64
 4   Tot_Suplr_Benes           548553 non-null   float64
 5   Tot_Suplr_Clms            1656449 non-null  float64
 6   Tot_Suplr_Srvcs           1656449 non-null  float64
 7   Avg_Suplr_Sbmtd_Chrg      1656449 non-null  float64
 8   Avg_Suplr_Mdcr_Alowd_Amt  1656449 non-null  float64
dtypes: float64(6), int64(1), object(2)
memory usage: 113.7+ MB


In [86]:
# count of npi values in DMEPOS dataset
print("Number of unique npi ids:", dmepos_df["Rfrg_NPI"].nunique())

Number of unique npi ids: 297769


In [87]:
dmepos_df["Rfrg_Prvdr_Gndr"].value_counts()

M    1075391
F     581012
Name: Rfrg_Prvdr_Gndr, dtype: int64

In [88]:
print("Number of unique provider type:", dmepos_df["Rfrg_Prvdr_Type"].nunique())

Number of unique provider type: 114


In [89]:
dmepos_df.head()

,Rfrg_NPI,Rfrg_Prvdr_Gndr,Rfrg_Prvdr_Type,Tot_Suplrs,Tot_Suplr_Benes,Tot_Suplr_Clms,Tot_Suplr_Srvcs,Avg_Suplr_Sbmtd_Chrg,Avg_Suplr_Mdcr_Alowd_Amt
0,1003000126,M,Internal Medicine,5.0,NaN,69.0,69.0,52.207536,22.961739
1,1003000126,M,Internal Medicine,5.0,12.0,88.0,88.0,362.305227,128.407955
2,1003000522,M,Family Practice,15.0,22.0,55.0,132.0,61.314242,8.364545
3,1003000522,M,Family Practice,2.0,NaN,13.0,13.0,14.158462,3.429231
4,1003000522,M,Family Practice,8.0,13.0,22.0,30.0,13.608667,1.436000


### Data Preprocessing

In [90]:
dmepos_features = dmepos_df.groupby(['Rfrg_NPI', 'Rfrg_Prvdr_Gndr', 'Rfrg_Prvdr_Type'])\
  .agg({'Tot_Suplrs':['mean'],
        'Tot_Suplr_Benes':['mean'],
        'Tot_Suplr_Clms':['mean'],
        'Tot_Suplr_Srvcs':['mean'],
        'Avg_Suplr_Sbmtd_Chrg':['mean'],
        'Avg_Suplr_Mdcr_Alowd_Amt': ['mean']})
dmepos_features.columns = [col[0]+"_Mean" for col in dmepos_features.columns.values]
dmepos_features = dmepos_features.reset_index(level=['Rfrg_NPI', 'Rfrg_Prvdr_Gndr', 'Rfrg_Prvdr_Type'])

In [91]:
dmepos_features = dmepos_features.merge(fraudster_list, how="left", left_on="Rfrg_NPI", right_on="npi")
dmepos_features["is_fraud"] = dmepos_features["is_fraud"].fillna(0)
dmepos_features = dmepos_features.rename(columns={"is_fraud": "Is_Fraud"})
dmepos_features = dmepos_features.drop(columns=["npi"])
dmepos_features = dmepos_features.rename(columns={"Rfrg_NPI": "NPI"})

In [92]:
print(dmepos_features["Is_Fraud"].value_counts())
print(dmepos_features["Is_Fraud"].value_counts(dropna=False, normalize=True))

0.0    297600
1.0       152
Name: Is_Fraud, dtype: int64
0.0    0.99949
1.0    0.00051
Name: Is_Fraud, dtype: float64


In [93]:
dmepos_features.head()

,NPI,Rfrg_Prvdr_Gndr,Rfrg_Prvdr_Type,Tot_Suplrs_Mean,Tot_Suplr_Benes_Mean,Tot_Suplr_Clms_Mean,Tot_Suplr_Srvcs_Mean,Avg_Suplr_Sbmtd_Chrg_Mean,Avg_Suplr_Mdcr_Alowd_Amt_Mean,Is_Fraud
0,1003000126,M,Internal Medicine,5.000000,12.0,78.500000,78.500000,207.256382,75.684847,0.0
1,1003000522,M,Family Practice,6.750000,17.5,25.750000,47.750000,124.364093,22.049944,0.0
2,1003000530,F,Internal Medicine,4.166667,NaN,13.000000,149.166667,43.711998,10.840529,0.0
3,1003000902,F,Family Practice,5.545455,16.5,20.545455,531.181818,57.993180,15.507345,0.0
4,1003001363,M,Pain Management,1.666667,18.0,18.000000,19.000000,1460.315913,763.323333,0.0


In [94]:
dmepos_features.isnull().sum()

NPI                                   0
Rfrg_Prvdr_Gndr                       0
Rfrg_Prvdr_Type                       0
Tot_Suplrs_Mean                       0
Tot_Suplr_Benes_Mean             151437
Tot_Suplr_Clms_Mean                   0
Tot_Suplr_Srvcs_Mean                  0
Avg_Suplr_Sbmtd_Chrg_Mean             0
Avg_Suplr_Mdcr_Alowd_Amt_Mean         0
Is_Fraud                              0
dtype: int64

In [95]:
dmepos_features["Tot_Suplr_Benes_Mean"].mean()

22.947904118596053

In [96]:
#Impute Missing Value with 5 and add a Tot_Suplr_Benes_mean_is_Missing flag
imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=5, add_indicator=True)
imputer = imputer.fit(dmepos_features[['Tot_Suplr_Benes_Mean']])
dmepos_features[['Tot_Suplr_Benes_Mean', 'Tot_Suplr_Benes_Mean_Missing']] = imputer.transform(dmepos_features[['Tot_Suplr_Benes_Mean']])

#OneHotEncode Rfrg_Prvdr_Gndr
dmepos_features['Is_Male'] = np.where(dmepos_features['Rfrg_Prvdr_Gndr']=='M', 1.0, 0.0)
dmepos_features = dmepos_features.drop(['Rfrg_Prvdr_Gndr'], axis=1)

dmepos_features.head()

,NPI,Rfrg_Prvdr_Type,Tot_Suplrs_Mean,Tot_Suplr_Benes_Mean,Tot_Suplr_Clms_Mean,Tot_Suplr_Srvcs_Mean,Avg_Suplr_Sbmtd_Chrg_Mean,Avg_Suplr_Mdcr_Alowd_Amt_Mean,Is_Fraud,Tot_Suplr_Benes_Mean_Missing,Is_Male
0,1003000126,Internal Medicine,5.000000,12.0,78.500000,78.500000,207.256382,75.684847,0.0,0.0,1.0
1,1003000522,Family Practice,6.750000,17.5,25.750000,47.750000,124.364093,22.049944,0.0,0.0,1.0
2,1003000530,Internal Medicine,4.166667,5.0,13.000000,149.166667,43.711998,10.840529,0.0,1.0,0.0
3,1003000902,Family Practice,5.545455,16.5,20.545455,531.181818,57.993180,15.507345,0.0,0.0,0.0
4,1003001363,Pain Management,1.666667,18.0,18.000000,19.000000,1460.315913,763.323333,0.0,0.0,1.0


### Descriptive Statistics

In [97]:
column_names = dmepos_features.drop(['NPI', 'Is_Fraud', 'Tot_Suplr_Benes_Mean_Missing', 'Is_Male', 'Rfrg_Prvdr_Type'], axis=1).columns.values.tolist()
descriptive_stats = dmepos_features.drop(['NPI', 'Is_Fraud', 'Tot_Suplr_Benes_Mean_Missing', 'Is_Male', 'Rfrg_Prvdr_Type'], axis=1).describe().loc[['mean', 'std', '50%']].transpose()
descriptive_stats.columns = ['mean', 'std', 'median']
descriptive_stats['mode'] = dmepos_features.drop(['NPI', 'Is_Fraud', 'Tot_Suplr_Benes_Mean_Missing', 'Is_Male', 'Rfrg_Prvdr_Type'], axis=1).mode().transpose()[0]
descriptive_stats['mad'] = [stats.median_abs_deviation(dmepos_features[x]) for x in column_names]
descriptive_stats['kurtosis'] = [stats.kurtosis(dmepos_features[x], fisher=False) for x in column_names]
descriptive_stats['excess kurtosis'] = [stats.kurtosis(dmepos_features[x], fisher=True) for x in column_names]
descriptive_stats['skewness'] = [stats.skew(dmepos_features[x]) for x in column_names]
descriptive_stats

,mean,std,median,mode,mad,kurtosis,excess kurtosis,skewness
Tot_Suplrs_Mean,3.926682,3.480310,3.000000,1.00,1.666667,25.201158,22.201158,3.209328
Tot_Suplr_Benes_Mean,13.819580,19.629373,5.000000,5.00,0.000000,827.194274,824.194274,16.777467
Tot_Suplr_Clms_Mean,27.391924,35.202689,20.000000,12.00,6.500000,391.744866,388.744866,13.211502
Tot_Suplr_Srvcs_Mean,514.810834,1956.421787,49.000000,12.00,36.000000,941.009003,938.009003,21.165150
Avg_Suplr_Sbmtd_Chrg_Mean,204.610679,480.382149,98.129805,302.91,57.985389,96.391263,93.391263,7.441320
Avg_Suplr_Mdcr_Alowd_Amt_Mean,83.699561,290.415856,29.933401,8.32,21.587946,110.817928,107.817928,9.397350


### Cleanup

In [70]:
del dmepos_df

## Part D

### Load Data

In [3]:
part_d_df = pd.read_csv("/content/drive/MyDrive/BT4012 Project/Submission Folder/Data_2019/Part_D_Providers_2019_NPI.csv", encoding='latin-1')

In [4]:
part_d_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1240595 entries, 0 to 1240594
Data columns (total 85 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          --------------    -----  
 0   PRSCRBR_NPI                     1240595 non-null  int64  
 1   Prscrbr_Last_Org_Name           1240595 non-null  object 
 2   Prscrbr_First_Name              1240594 non-null  object 
 3   Prscrbr_MI                      837420 non-null   object 
 4   Prscrbr_Crdntls                 1151088 non-null  object 
 5   Prscrbr_Gndr                    1240594 non-null  object 
 6   Prscrbr_Ent_Cd                  1240595 non-null  object 
 7   Prscrbr_St1                     1240593 non-null  object 
 8   Prscrbr_St2                     369829 non-null   object 
 9   Prscrbr_City                    1240594 non-null  object 
 10  Prscrbr_State_Abrvtn            1240595 non-null  object 
 11  Prscrbr_State_FIPS              1239380 non-null  float64
 12  

In [5]:
print("Number of unique Prescriber NPIs:", part_d_df["PRSCRBR_NPI"].nunique())

Number of unique Prescriber NPIs: 1240595


In [6]:
part_d_df.head()

,PRSCRBR_NPI,Prscrbr_Last_Org_Name,Prscrbr_First_Name,Prscrbr_MI,Prscrbr_Crdntls,Prscrbr_Gndr,Prscrbr_Ent_Cd,Prscrbr_St1,Prscrbr_St2,Prscrbr_City,...,Bene_Male_Cnt,Bene_Race_Wht_Cnt,Bene_Race_Black_Cnt,Bene_Race_Api_Cnt,Bene_Race_Hspnc_Cnt,Bene_Race_Natind_Cnt,Bene_Race_Othr_Cnt,Bene_Dual_Cnt,Bene_Ndual_Cnt,Bene_Avg_Risk_Scre
0,1003000126,Enkeshafi,Ardalan,NaN,M.D.,M,I,900 Seton Dr,NaN,Cumberland,...,96.0,130.0,73.0,NaN,NaN,0.0,NaN,56.0,158.0,2.708114
1,1003000142,Khalil,Rashid,NaN,M.D.,M,I,4126 N Holland Sylvania Rd,Suite 220,Toledo,...,96.0,162.0,92.0,NaN,19.0,0.0,NaN,157.0,125.0,1.856453
2,1003000167,Escobar,Julio,E,DDS,M,I,5 Pine Cone Rd,NaN,Dayton,...,21.0,48.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,1.140737
3,1003000175,Reyes-Vasquez,Belinda,NaN,D.D.S.,F,I,322 N Azusa Ave Ste 202,NaN,La Puente,...,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.764182
4,1003000282,Blakemore,Rosie,K,FNP,F,I,Tennessee Prison For Women,3881 Stewarts Lane,Nashville,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.156650


In [7]:
part_d_df.columns

Index(['PRSCRBR_NPI', 'Prscrbr_Last_Org_Name', 'Prscrbr_First_Name',
       'Prscrbr_MI', 'Prscrbr_Crdntls', 'Prscrbr_Gndr', 'Prscrbr_Ent_Cd',
       'Prscrbr_St1', 'Prscrbr_St2', 'Prscrbr_City', 'Prscrbr_State_Abrvtn',
       'Prscrbr_State_FIPS', 'Prscrbr_zip5', 'Prscrbr_RUCA',
       'Prscrbr_RUCA_Desc', 'Prscrbr_Cntry', 'Prscrbr_Type',
       'Prscrbr_Type_src', 'Tot_Clms', 'Tot_30day_Fills', 'Tot_Drug_Cst',
       'Tot_Day_Suply', 'Tot_Benes', 'GE65_Sprsn_Flag', 'GE65_Tot_Clms',
       'GE65_Tot_30day_Fills', 'GE65_Tot_Drug_Cst', 'GE65_Tot_Day_Suply',
       'GE65_Bene_Sprsn_Flag', 'GE65_Tot_Benes', 'Brnd_Sprsn_Flag',
       'Brnd_Tot_Clms', 'Brnd_Tot_Drug_Cst', 'Gnrc_Sprsn_Flag',
       'Gnrc_Tot_Clms', 'Gnrc_Tot_Drug_Cst', 'Othr_Sprsn_Flag',
       'Othr_Tot_Clms', 'Othr_Tot_Drug_Cst', 'MAPD_Sprsn_Flag',
       'MAPD_Tot_Clms', 'MAPD_Tot_Drug_Cst', 'PDP_Sprsn_Flag', 'PDP_Tot_Clms',
       'PDP_Tot_Drug_Cst', 'LIS_Sprsn_Flag', 'LIS_Tot_Clms', 'LIS_Drug_Cst',
       'NonLIS_Sprsn_

### Data Preprocessing

In [13]:
# Merge with Fraudster List 
part_d_df = part_d_df.merge(fraudster_list, how="left", left_on="PRSCRBR_NPI", right_on="npi")
part_d_df["is_fraud"] = part_d_df["is_fraud"].fillna(0)
part_d_df = part_d_df.rename(columns={"is_fraud": "Is_Fraud"})
part_d_df = part_d_df.drop(columns=["npi"])
#part_d_df = part_d_df.rename(columns={"PRSCRBR_NPI": "NPI"})

In [14]:
print(part_d_df["Is_Fraud"].value_counts())
print(part_d_df["Is_Fraud"].value_counts(dropna=False, normalize=True))

0.0    1240132
1.0        465
Name: Is_Fraud, dtype: int64
0.0    0.999625
1.0    0.000375
Name: Is_Fraud, dtype: float64


### Variables Supression

According the the data source, values for some columns are supressed due to privacy concerns if they are small
- For Beneficiary counts, either flag is given (but reason unknown) OR no flag and NaN if beneficiares <= 10. 
- For Cost, flag is given but (but reason unknown)

Example below

In [15]:
part_d_df[['Opioid_Tot_Clms', 'Opioid_Tot_Drug_Cst', 'Opioid_Tot_Suply', 'Opioid_Tot_Benes']]

,Opioid_Tot_Clms,Opioid_Tot_Drug_Cst,Opioid_Tot_Suply,Opioid_Tot_Benes
0,30.0,216.52,159.0,29.0
1,464.0,15671.07,12567.0,173.0
2,NaN,NaN,NaN,NaN
3,0.0,0.00,0.0,0.0
4,0.0,0.00,0.0,NaN
...,...,...,...,...
1240592,26.0,346.15,540.0,15.0
1240593,12.0,104.24,57.0,NaN
1240594,29.0,267.93,135.0,28.0
1240595,0.0,0.00,0.0,NaN


_Sprsn_Flag column indicates that corresponding values have been supressed. NaN's means values exist. As reasoning is not given for supression, we change the character flag into binary flag

In [43]:
part_d_df_x = part_d_df.copy()

In [44]:
part_d_df_x = part_d_df_x.rename(columns={"PRSCRBR_NPI": "NPI"})

In [45]:
# Encoding Supression Variables into Binary 
agg_by = {'NPI':["count"]}
for col in part_d_df_x.columns:
  if col.endswith("_Sprsn_Flag"):
    part_d_df_x[col] = np.where(part_d_df_x[col].isna(), 0, 1)
    agg_by[col] = ["sum"]

#### Drop columns
Provider Static information and sub-group beneficiary counts & beneificiary demographic 

In [46]:
part_d_df_x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1240597 entries, 0 to 1240596
Data columns (total 86 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          --------------    -----  
 0   NPI                             1240597 non-null  int64  
 1   Prscrbr_Last_Org_Name           1240597 non-null  object 
 2   Prscrbr_First_Name              1240596 non-null  object 
 3   Prscrbr_MI                      837422 non-null   object 
 4   Prscrbr_Crdntls                 1151090 non-null  object 
 5   Prscrbr_Gndr                    1240596 non-null  object 
 6   Prscrbr_Ent_Cd                  1240597 non-null  object 
 7   Prscrbr_St1                     1240595 non-null  object 
 8   Prscrbr_St2                     369831 non-null   object 
 9   Prscrbr_City                    1240596 non-null  object 
 10  Prscrbr_State_Abrvtn            1240597 non-null  object 
 11  Prscrbr_State_FIPS              1239382 non-null  float64
 12  

In [47]:
part_d_to_drop_columns = []
for col in part_d_df_x.columns:
  if col.endswith("_Tot_Benes") or col.endswith("_Bene_Sprsn_Flag"):
    part_d_to_drop_columns.append(col)
    continue
  if col.startswith("Bene_"):
    part_d_to_drop_columns.append(col)
    continue
  if col.startswith("Prscrbr_"):
    part_d_to_drop_columns.append(col)

In [48]:
part_d_df_x = part_d_df_x.drop(columns= part_d_to_drop_columns)

In [49]:
part_d_df_x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1240597 entries, 0 to 1240596
Data columns (total 47 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          --------------    -----  
 0   NPI                             1240597 non-null  int64  
 1   Tot_Clms                        1240597 non-null  int64  
 2   Tot_30day_Fills                 1240597 non-null  float64
 3   Tot_Drug_Cst                    1240597 non-null  float64
 4   Tot_Day_Suply                   1240597 non-null  int64  
 5   Tot_Benes                       1107639 non-null  float64
 6   GE65_Sprsn_Flag                 1240597 non-null  int64  
 7   GE65_Tot_Clms                   957060 non-null   float64
 8   GE65_Tot_30day_Fills            957060 non-null   float64
 9   GE65_Tot_Drug_Cst               957060 non-null   float64
 10  GE65_Tot_Day_Suply              957060 non-null   float64
 11  Brnd_Sprsn_Flag                 1240597 non-null  int64  
 12  

#### Correlation with target

In [52]:
print("-- Correlation of Part D Features with Fraud --")
for col, correl in part_d_df_x.corrwith(part_d_df_x["Is_Fraud"]).abs().sort_values(ascending=False).items():
  if (col == "Is_Fraud"):
    continue
  print(col, correl)

-- Correlation of Part D Features with Fraud --
Opioid_Tot_Drug_Cst 0.012674546889685638
Opioid_Tot_Suply 0.011641720257233009
Opioid_Tot_Clms 0.011333747056182017
Opioid_LA_Prscrbr_Rate 0.010608107102583125
Opioid_LA_Tot_Drug_Cst 0.010075201345634525
LIS_Drug_Cst 0.010008859644704186
Gnrc_Tot_Drug_Cst 0.009922107416735165
Opioid_LA_Tot_Clms 0.009373728166976344
Opioid_LA_Tot_Suply 0.00919984939608887
Antbtc_Tot_Drug_Cst 0.008951330050356041
LIS_Tot_Clms 0.007271755318029846
Antpsyct_GE65_Tot_Clms 0.006874035127909062
PDP_Tot_Drug_Cst 0.005860173871250088
Brnd_Tot_Drug_Cst 0.005315222835510263
Tot_Drug_Cst 0.004952275058838614
Gnrc_Tot_Clms 0.004826962149402464
PDP_Tot_Clms 0.004747055157348083
Tot_Clms 0.004552398391442056
MAPD_Tot_Clms 0.0036167828922376542
Brnd_Tot_Clms 0.0035235955444470595
GE65_Tot_Drug_Cst 0.0033310123844188947
MAPD_Tot_Drug_Cst 0.003021471580035755
Othr_Tot_Clms 0.0025558184299585267
Antbtc_Tot_Clms 0.0025231831567833094
GE65_Tot_Clms 0.0024832395406679964
Tot_3

### Keep only columns of interest

In [53]:
part_d_df_x = part_d_df_x[["NPI", 'Tot_Clms', 'Tot_30day_Fills', 'Tot_Drug_Cst', "Tot_Day_Suply", 'Tot_Benes',
                           "Opioid_Tot_Clms", "Opioid_Tot_Drug_Cst", "Opioid_Tot_Suply", 
                           "Opioid_LA_Tot_Clms", "Opioid_LA_Tot_Drug_Cst", "Opioid_LA_Tot_Suply", 
                           'Is_Fraud']]

In [54]:
part_d_df_x.head()

,NPI,Tot_Clms,Tot_30day_Fills,Tot_Drug_Cst,Tot_Day_Suply,Tot_Benes,Opioid_Tot_Clms,Opioid_Tot_Drug_Cst,Opioid_Tot_Suply,Opioid_LA_Tot_Clms,Opioid_LA_Tot_Drug_Cst,Opioid_LA_Tot_Suply,Is_Fraud
0,1003000126,589,681.733333,28902.12,15955,214.0,30.0,216.52,159.0,NaN,NaN,NaN,0.0
1,1003000142,1092,1156.166667,61987.88,31971,282.0,464.0,15671.07,12567.0,71.0,8008.12,1924.0,0.0
2,1003000167,73,73.000000,370.56,530,52.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,1003000175,21,21.000000,176.64,162,11.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0
4,1003000282,63,91.000000,2221.14,2693,NaN,0.0,0.00,0.0,0.0,0.00,0.0,0.0


### Inputation and Feature Engineering

In [55]:
part_d_df_x.isnull().sum()

NPI                            0
Tot_Clms                       0
Tot_30day_Fills                0
Tot_Drug_Cst                   0
Tot_Day_Suply                  0
Tot_Benes                 132958
Opioid_Tot_Clms           346236
Opioid_Tot_Drug_Cst       346236
Opioid_Tot_Suply          346236
Opioid_LA_Tot_Clms        441904
Opioid_LA_Tot_Drug_Cst    441904
Opioid_LA_Tot_Suply       441904
Is_Fraud                       0
dtype: int64

In [29]:
# Impute Number of Beneficiary - use 5 as recommended by cms  
part_d_df_x["Tot_Benes"] = np.where(part_d_df_x["Tot_Benes"].isna(), 5, part_d_df_x["Tot_Benes"])
# Impute number of Opiod Claims & add missing flag 
part_d_df_x["Opioid_Tot_Clms_Missing"] = np.where(part_d_df_x["Opioid_Tot_Clms"].isna(), 1, 0)
part_d_df_x["Opioid_Tot_Clms"] = np.where(part_d_df_x["Opioid_Tot_Clms"].isna(), 5, part_d_df_x["Opioid_Tot_Clms"])
# Impute number of LA Opiod Claims 
part_d_df_x["Opioid_LA_Tot_Clms_Missing"] = np.where(part_d_df_x["Opioid_LA_Tot_Clms"].isna(), 1, 0)
part_d_df_x["Opioid_LA_Tot_Clms"] = np.where(part_d_df_x["Opioid_LA_Tot_Clms"].isna(), 5, part_d_df_x["Opioid_LA_Tot_Clms"])

In [30]:
# Average Drug Cost per claim: Total Drug Cost / Total Claims
part_d_df_x["Avg_Clm_Drug_Cost"] = part_d_df_x["Tot_Drug_Cst"] / part_d_df_x["Tot_Clms"]
# Average Drug Cost per beneficiary : Total Drug Cost / Total Beneficiarys
part_d_df_x["Avg_Bene_Cost"] = part_d_df_x["Tot_Drug_Cst"] / part_d_df_x["Tot_Benes"]
# Average Number of Claims per beneficiary 
part_d_df_x["Avg_Bene_Clms"] = part_d_df_x["Tot_Clms"] / part_d_df_x["Tot_Benes"]

In [31]:
# Percentage Opiod claims
part_d_df_x["Pct_Opioid_Clms"] = part_d_df_x["Opioid_Tot_Clms"] / part_d_df_x["Tot_Clms"]
part_d_df_x["Pct_Opioid_LA_Clms"] = part_d_df_x["Opioid_LA_Tot_Clms"] / part_d_df_x["Tot_Clms"]

# Combine

### Merge Dataset

In [98]:
# Add binary indicator variables
dmepos_features["In_Dmepos"] = 1 
part_d_df_x["In_Partd"] =  1

In [99]:
# Drop the Rfrg_Prvdr_Type and is_male from dmepos and take reference from part_b
combined_dataset = part_b_features.merge(dmepos_features.drop(columns=["Is_Fraud", "Rfrg_Prvdr_Type", "Is_Male"]), how="inner", on="NPI")
combined_dataset = combined_dataset.merge(part_d_df_x.drop(columns=["Is_Fraud"]), how="inner", on="NPI", copy=False)

In [100]:
len(combined_dataset)

263438

In [ ]:
combined_dataset["Is_Fraud"].value_counts()

0.0    263332
1.0       106
Name: Is_Fraud, dtype: int64

In [ ]:
combined_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 263438 entries, 0 to 263437
Data columns (total 37 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   NPI                            263438 non-null  int64  
 1   Rndrng_Prvdr_Type              263438 non-null  object 
 2   Tot_Benes_Mean                 263438 non-null  Float64
 3   Tot_Srvcs_Mean                 263438 non-null  float64
 4   Tot_Bene_Day_Srvcs_Mean        263438 non-null  Float64
 5   Avg_Sbmtd_Chrg_Mean            263438 non-null  float64
 6   Avg_Mdcr_Alowd_Amt_Mean        263438 non-null  float64
 7   Avg_Mdcr_Pymt_Amt_Mean         263438 non-null  float64
 8   Is_Fraud                       263438 non-null  float64
 9   Is_Male                        263438 non-null  float64
 10  Tot_Suplrs_Mean                263438 non-null  float64
 11  Tot_Suplr_Benes_Mean           263438 non-null  float64
 12  Tot_Suplr_Clms_Mean           

In [ ]:
print(combined_dataset.groupby("Is_Fraud")["In_Dmepos"].value_counts())
print(combined_dataset.groupby("Is_Fraud")["In_Partd"].value_counts())

Is_Fraud  In_Dmepos
0.0       1            263332
1.0       1               106
Name: In_Dmepos, dtype: int64
Is_Fraud  In_Partd
0.0       1           263332
1.0       1              106
Name: In_Partd, dtype: int64


In [ ]:
Rndrng_Prvdr_Type_dummies = pd.get_dummies(combined_dataset['Rndrng_Prvdr_Type'])
combined_dataset = pd.concat([combined_dataset.drop(columns=["Rndrng_Prvdr_Type"]), Rndrng_Prvdr_Type_dummies], axis=1)
combined_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 263438 entries, 0 to 263437
Columns: 108 entries, NPI to Vascular Surgery
dtypes: Float64(2), float64(27), int64(7), uint8(72)
memory usage: 93.0 MB


In [ ]:
combined_dataset.head()

,NPI,Tot_Benes_Mean,Tot_Srvcs_Mean,Tot_Bene_Day_Srvcs_Mean,Avg_Sbmtd_Chrg_Mean,Avg_Mdcr_Alowd_Amt_Mean,Avg_Mdcr_Pymt_Amt_Mean,Is_Fraud,Is_Male,Tot_Suplrs_Mean,...,Radiation Oncology,Rheumatology,Sleep Medicine,Sports Medicine,Surgical Oncology,Thoracic Surgery,Undefined Physician type,Undersea and Hyperbaric Medicine,Urology,Vascular Surgery
0,1003000126,107.666667,151.888889,151.888889,420.854343,126.457125,100.817897,0.0,1.0,5.000000,...,0,0,0,0,0,0,0,0,0,0
1,1003000522,66.4,80.466667,80.466667,241.739167,77.594820,62.790339,0.0,1.0,6.750000,...,0,0,0,0,0,0,0,0,0,0
2,1003000530,60.6875,88.250000,88.25,168.426163,107.909471,94.896525,0.0,0.0,4.166667,...,0,0,0,0,0,0,0,0,0,0
3,1003000902,49.384615,72.230769,72.230769,125.538462,62.654035,50.587626,0.0,0.0,5.545455,...,0,0,0,0,0,0,0,0,0,0
4,1003001363,124.678571,590.678571,242.464286,282.029535,133.812472,102.774391,0.0,1.0,1.666667,...,0,0,0,0,0,0,0,0,0,0


### Cleanup

In [ ]:
del dmepos_features
del part_d_df_x

## Train Test Split

In [ ]:
train_df, test_df = train_test_split(combined_dataset, test_size=0.2, random_state=42, stratify=combined_dataset["Is_Fraud"])

In [ ]:
print(train_df["Is_Fraud"].value_counts())
print(test_df["Is_Fraud"].value_counts())

0.0    210665
1.0        85
Name: Is_Fraud, dtype: int64
0.0    52667
1.0       21
Name: Is_Fraud, dtype: int64


## Imputation

In [ ]:
print(train_df.columns[train_df.isnull().any()].tolist())
print(test_df.columns[test_df.isnull().any()].tolist())

['Opioid_Tot_Drug_Cst', 'Opioid_Tot_Suply', 'Opioid_LA_Tot_Drug_Cst', 'Opioid_LA_Tot_Suply']
['Opioid_Tot_Drug_Cst', 'Opioid_Tot_Suply', 'Opioid_LA_Tot_Drug_Cst', 'Opioid_LA_Tot_Suply']


In [ ]:
# Impute missing Opioid_Tot_Drug_Cst with median
imputer = SimpleImputer(missing_values=np.nan, strategy='median', add_indicator=True)
imputer = imputer.fit(train_df[['Opioid_Tot_Drug_Cst']])
train_df[['Opioid_Tot_Drug_Cst', 'Opioid_Tot_Drug_Cst_Missing']] = imputer.transform(train_df[['Opioid_Tot_Drug_Cst']])
test_df[['Opioid_Tot_Drug_Cst', 'Opioid_Tot_Drug_Cst_Missing']] = imputer.transform(test_df[['Opioid_Tot_Drug_Cst']])

In [ ]:
# Impute missing Opioid_Tot_Suply with median
imputer = SimpleImputer(missing_values=np.nan, strategy='median', add_indicator=True)
imputer = imputer.fit(train_df[['Opioid_Tot_Suply']])
train_df[['Opioid_Tot_Suply', 'Opioid_Tot_Suply_Missing']] = imputer.transform(train_df[['Opioid_Tot_Suply']])
test_df[['Opioid_Tot_Suply', 'Opioid_Tot_Suply_Missing']] = imputer.transform(test_df[['Opioid_Tot_Suply']])

In [ ]:
# Impute missing Opioid_LA_Tot_Drug_Cst with median
imputer = SimpleImputer(missing_values=np.nan, strategy='median', add_indicator=True)
imputer = imputer.fit(train_df[['Opioid_LA_Tot_Drug_Cst']])
train_df[['Opioid_LA_Tot_Drug_Cst', 'Opioid_LA_Tot_Drug_Cst_Missing']] = imputer.transform(train_df[['Opioid_LA_Tot_Drug_Cst']])
test_df[['Opioid_LA_Tot_Drug_Cst', 'Opioid_LA_Tot_Drug_Cst_Missing']] = imputer.transform(test_df[['Opioid_LA_Tot_Drug_Cst']])

In [ ]:
# Impute missing Opioid_LA_Tot_Suply with median
imputer = SimpleImputer(missing_values=np.nan, strategy='median', add_indicator=True)
imputer = imputer.fit(train_df[['Opioid_LA_Tot_Suply']])
train_df[['Opioid_LA_Tot_Suply', 'Opioid_LA_Tot_Suply_Missing']] = imputer.transform(train_df[['Opioid_LA_Tot_Suply']])
test_df[['Opioid_LA_Tot_Suply', 'Opioid_LA_Tot_Suply_Missing']] = imputer.transform(test_df[['Opioid_LA_Tot_Suply']])

In [ ]:
print(train_df.columns[train_df.isnull().any()].tolist())
print(test_df.columns[test_df.isnull().any()].tolist())

[]
[]


## Export

In [ ]:
train_df.to_csv("/content/drive/MyDrive/BT4012 Project/Submission Folder/train_df.csv", index=False)
test_df.to_csv("/content/drive/MyDrive/BT4012 Project/Submission Folder/test_df.csv", index=False)